In [1]:
import pandas as pd
import numpy as np
import spacy
#this loads the wrapper
#nlp = spacy.load('en_core_web_sm')
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
import nltk
from scipy import spatial

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample = pd.read_csv('sample_submission.csv')

In [3]:
test.shape

(36, 4)

In [4]:
train

,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00
...,...,...,...,...,...
36468,8e1386cbefd7f245,wood article,wooden article,B44,1.00
36469,42d9e032d1cd3242,wood article,wooden box,B44,0.50
36470,208654ccb9e14fa3,wood article,wooden handle,B44,0.50
36471,756ec035e694722b,wood article,wooden material,B44,0.75


In [5]:
sentences = [word_tokenize(i) for i in list(train.target)]

In [6]:
model = Word2Vec(sentences)

In [7]:
class MyTokenizer:
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        transformed_X = []
        for document in X:
            tokenized_doc = []
            for sent in nltk.sent_tokenize(document):
                tokenized_doc += nltk.word_tokenize(sent)
            transformed_X.append(np.array(tokenized_doc))
        return np.array(transformed_X)
    
    def fit_transform(self, X, y=None):
        return self.transform(X)

class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(word2vec.wv.syn0[0])

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = MyTokenizer().fit_transform(X)
        
        return np.array([
            np.mean([self.word2vec.wv[w] for w in words if w in self.word2vec.wv]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])
    
    def fit_transform(self, X, y=None):
        return self.transform(X)

In [8]:
mean_embedding_vectorizer = MeanEmbeddingVectorizer(model)
mean_embedded = mean_embedding_vectorizer.fit_transform(train['target'])

/Users/robinmasliah/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
/Users/robinmasliah/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  from ipykernel import kernelapp as app


In [9]:
test_target = mean_embedding_vectorizer.fit_transform(test['target'])

/Users/robinmasliah/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  from ipykernel import kernelapp as app


In [10]:
test_anchor = mean_embedding_vectorizer.fit_transform(test['anchor'])

/Users/robinmasliah/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  from ipykernel import kernelapp as app


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

v = TfidfVectorizer()
x = v.fit_transform(test['target'])

In [12]:
A = test_anchor[0]
B = test_target[13]

In [14]:
def cosine_similarity_vectors(a, b):
    cosine_similarity = 1 - spatial.distance.cosine(a, b)
    return cosine_similarity

In [38]:
test_target = pd.DataFrame({'test_target': test_target.tolist()})
test_anchor = pd.DataFrame({'test_anchor': test_anchor.tolist()})
_id = test['id']
frames = [_id, test_anchor, test_target]
result = pd.concat(frames, axis=1)

In [39]:
result['test_target'] = result['test_target'].apply(lambda x: np.array(x))
result['test_anchor'] = result['test_anchor'].apply(lambda x: np.array(x))

In [40]:
result['score'] = 0
result["score"] = result.apply(lambda x: cosine_similarity_vectors(x["test_anchor"], x["test_target"]), axis=1)

/Users/robinmasliah/opt/anaconda3/lib/python3.7/site-packages/scipy/spatial/distance.py:714: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [41]:
result[['id', 'score']]

,id,score
0,4112d61851461f60,0.998691
1,09e418c93a776564,0.999910
2,36baf228038e314b,0.999553
3,1f37ead645e7f0c8,0.999142
4,71a5b6ad068d531f,0.998416
5,474c874d0c07bd21,0.997387
6,442c114ed5c4e3c9,0.996508
7,b8ae62ea5e1d8bdb,0.999569
8,faaddaf8fcba8a3f,0.999601
9,ae0262c02566d2ce,0.999809


In [48]:
test['score'] = 0

In [49]:
test["score"] = test.apply(lambda x: coeff_similarity(x["anchor"], x["target"]), axis=1)

/Users/robinmasliah/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  """


In [50]:
test[['score']].to_csv('sample_robin_submission.csv')